In [1]:
!pip install langchain langchain-community pypdf

Defaulting to user installation because normal site-packages is not writeable


In [9]:
pip install faiss-cpu


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   --- ------------------------------------ 1.3/14.9 MB 9.6 MB/s eta 0:00:02
   ------------------- -------------------- 7.3/14.9 MB 22.7 MB/s eta 0:00:01
   --------------------------- ------------ 10.2/14.9 MB 23.6 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 21.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install langchain-ollama


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from operator import itemgetter

In [2]:
PDF_FILE = "PEFT_LORA.pdf" # leme load the novel first
MODEL = "llama3.2:1b" # on the next step we we will load the model

In [3]:

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content)

Number of pages: 28
Length of a page: 879
Content of a page: PEFT
2This keeps the number of trainable p arameters low without sacr ificing 
performance.
Quick Mat h R ec ap:
W  W ₀ + ΔW
ΔW  BA
W is the updat ed weight matr ix, and W ₀ is the or iginal one. B and A ar e the 
low-rank matr ices that r epresent the updat e.
In simple t erms, instead of updating all the p arameters of a w eight matr ix, LoRA  
uses smaller matr ices to make targeted adjustment s, reducing the comput ation 
load.
import numpy as np
# Original weight matrix
W0 = np.array([[1, 2, 3],
                [4, 5, 6]])
# Low-rank matrices
A = np.array([[0.1, 0.2]])  # Shape : (1, 2)
B = np.array([[0.3],
              [0.4],
              [0.5]])  # Shape : (3, 1)
# LoRA update
delta_W = np.dot(A.T, B.T)
# Final adapted weight
W = W0 + delta_W
print("Original W0:" )
print(W0)
print("\nLoRA update (ΔW):" )
print(delta_W)
print("\nFinal adapted W:" )
print(W)


In [4]:
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=150)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)


Number of chunks: 28
Length of a chunk: 879
Content of a chunk: PEFT
2This keeps the number of trainable p arameters low without sacr ificing 
performance.
Quick Mat h R ec ap:
W  W ₀ + ΔW
ΔW  BA
W is the updat ed weight matr ix, and W ₀ is the or iginal one. B and A ar e the 
low-rank matr ices that r epresent the updat e.
In simple t erms, instead of updating all the p arameters of a w eight matr ix, LoRA  
uses smaller matr ices to make targeted adjustment s, reducing the comput ation 
load.
import numpy as np
# Original weight matrix
W0 = np.array([[1, 2, 3],
                [4, 5, 6]])
# Low-rank matrices
A = np.array([[0.1, 0.2]])  # Shape : (1, 2)
B = np.array([[0.3],
              [0.4],
              [0.5]])  # Shape : (3, 1)
# LoRA update
delta_W = np.dot(A.T, B.T)
# Final adapted weight
W = W0 + delta_W
print("Original W0:" )
print(W0)
print("\nLoRA update (ΔW):" )
print(delta_W)
print("\nFinal adapted W:" )
print(W)


In [5]:


embeddings = OllamaEmbeddings(model=MODEL)
vectorstore = FAISS.from_documents(chunks, embeddings) # storein vector store

C:\Users\Kiyo\AppData\Local\Temp\ipykernel_20212\3103334380.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL)


In [6]:
retriever = vectorstore.as_retriever()
retriever.invoke("What is LORA?")

[Document(metadata={'source': 'PEFT_LORA.pdf', 'page': 5}, page_content='PEFT\n6import numpy as np\n# Original activations\nx = np.array([[1, 2, 3],\n              [4, 5, 6]])\n# Learned rescaling vector\nl = np.array([0.5, 1.5, 1.0])\n# IA³ rescaling\ny = x * l  # Element -wise multiplication\nprint("Original activations:" )\nprint(x)\nprint("\\nRescaling vector:" )\nprint(l)\nprint("\\nRescaled activations:" )\nprint(y)\nOriginal activations :\n[[1 2 3]\n [4 5 6]]\nRescaling vector :\n[0.5 1.5 1. ]\nRescaled activations :\n[[0.5 3.  3. ]\n [2.  7.5 6. ]]\nWhat ʼ s happening?\nWe have some activ ations \nx that we want t o modify . Instead of changing them all equall y, we use a \nlearned vector l to scale them dif ferently. This wa y, some p arts of the model  \nget more or less at tention, which helps it adapt bet ter to the task at hand.\n4 .  L o H a  ( L o w - R a n k  H a d a m a r d  P r o d u c t ) :'),
 Document(metadata={'source': 'PEFT_LORA.pdf', 'page': 4}, page_content="P

In [7]:
retriever = vectorstore.as_retriever()
retriever.invoke("Explain LORA types?")

[Document(metadata={'source': 'PEFT_LORA.pdf', 'page': 5}, page_content='PEFT\n6import numpy as np\n# Original activations\nx = np.array([[1, 2, 3],\n              [4, 5, 6]])\n# Learned rescaling vector\nl = np.array([0.5, 1.5, 1.0])\n# IA³ rescaling\ny = x * l  # Element -wise multiplication\nprint("Original activations:" )\nprint(x)\nprint("\\nRescaling vector:" )\nprint(l)\nprint("\\nRescaled activations:" )\nprint(y)\nOriginal activations :\n[[1 2 3]\n [4 5 6]]\nRescaling vector :\n[0.5 1.5 1. ]\nRescaled activations :\n[[0.5 3.  3. ]\n [2.  7.5 6. ]]\nWhat ʼ s happening?\nWe have some activ ations \nx that we want t o modify . Instead of changing them all equall y, we use a \nlearned vector l to scale them dif ferently. This wa y, some p arts of the model  \nget more or less at tention, which helps it adapt bet ter to the task at hand.\n4 .  L o H a  ( L o w - R a n k  H a d a m a r d  P r o d u c t ) :'),
 Document(metadata={'source': 'PEFT_LORA.pdf', 'page': 4}, page_content="P

In [11]:
model = ChatOllama(model=MODEL, temperature=0)
model.invoke("What LORA types you know?")

AIMessage(content='I can provide information on various types of Low-Rate Audio (LORA) systems. Here are some examples:\n\n1. **LORAN C**: A navigation system that uses a network of radio transmitters and receivers to provide location information.\n2. **LORAN A**: An improved version of the LORAN C system, which provides more accurate location information.\n3. **LORAN V**: The latest generation of the LORAN system, which offers even better accuracy and reliability.\n4. **LORAN C1**: A variant of the LORAN C system that uses a single transmitter to provide location information.\n5. **LORAN C2**: An improved version of the LORAN C system that provides more accurate location information.\n\nIn terms of specific applications, LORA systems are often used in:\n\n1. **Navigation and mapping**: LORAN is commonly used in navigation systems such as GPS (Global Positioning System) and GLONASS.\n2. **Emergency services**: LORAN is used by emergency services such as ambulance and fire departments t

In [12]:


parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("What LORA types you know?"))

I can provide information on various types of Low-Rate Area Network (LRAN) technologies. Here are some of the most well-known ones:

1. **LORAN-C**: A satellite-based navigation system that uses a network of satellites to provide location information and timing signals.
2. **LORAN-A**: An older version of LORAN, which was used until 2006. It's still in use today, but it has been largely replaced by more modern systems.
3. **LORAN-C/CRS (Cesium-Radio Star)**: A variant of LORAN that uses cesium atomic clocks to provide more accurate location information.
4. **LORAN-C/CRS-2**: An updated version of the CRS system, which offers improved accuracy and reliability.
5. **LORAN-C/CRS-3**: The latest version of the CRS system, which provides even higher accuracy and reliability than its predecessors.

These are some of the most common types of Loran systems used in navigation applications. There may be other variants or specialized systems as well.


In [13]:


template = """
You are an assistant that provides answers to questions based on
a given information. 

You need to answer the question based on the given information If you can't answer the
question, reply "I don't know pallll".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


You are an assistant that provides answers to questions based on
a given information. 

You need to answer the question based on the given information If you can't answer the
question, reply "I don't know pallll".

Be as concise as possible and go straight to the point.

Context: Here is some context

Question: Here is a question



In [15]:
chain = prompt | model | parser

chain.invoke({
    "context": "LoRA", 
    "question": "Explain LoRA types?"
})


'LoRaWAN is a wireless communication technology used for IoT (Internet of Things) applications. There are three main LoRaWAN types:\n\n1. **LoRaWAN Classic**: The original version, using 868 MHz frequency band.\n2. **LoRaWAN Cat B20**: Using 868 MHz and 915 MHz frequencies, offering better range and performance.\n3. **LoRaWAN Cat NB**: Using 868 MHz and 915 MHz frequencies, optimized for low-power applications.\n\nEach type has its own characteristics, advantages, and use cases.'

In [16]:


chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [18]:
questions = [
    "Give information on Parameter-Efficient Fine-Tuning",

]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")

Question: Give information on Parameter-Efficient Fine-Tuning
Answer: Parameter-Efficient Fine-Tuning (PEFT) is a technique used in deep learning models to fine-tune them on specific tasks or datasets without increasing their computational requirements. Here are some key points about PEFT:

1. **Principle**: PEFT involves using learned vectors, called "IA³" (Inferred Activation 3), to scale model activations and adapt the weights of a pre-trained model.
2. **Advantages**:
	* Reduces computational requirements by only scaling specific parts of the model.
	* Can be used for mixed-task models where different tasks require different adaptations.
	* Allows for more efficient training on smaller datasets or with lower computational resources.
3. **Components**: PEFT typically involves three main components:
	* IA³: A learned vector that scales activations in self-attention and feed-forward layers.
	* Learned vectors (L): Pre-trained weights of the model used to scale activations.
4. **Traini